In [ ]:
# Import libraries

import pandas as pd
import numpy as np
import os

In [22]:
os.chdir('c:\\Users\\henry\\OneDrive\\Personal Career\\Personal Projects\\GitHub\\Glacier-Product-Release-Impact-Analysis\\raw_data')

accounts = pd.read_csv("ravenstack_accounts.csv")
users = pd.read_csv("customer_churn_business_dataset.csv")

In [23]:
accounts["account_id"]

0      A-2e4581
1      A-43a9e3
2      A-0a282f
3      A-1f0ac7
4      A-ce550d
         ...   
495    A-8ae3fc
496    A-55f257
497    A-d26ab4
498    A-712533
499    A-781cc0
Name: account_id, Length: 500, dtype: object